In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, UpSampling2D, Dense, Flatten # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image

# Constants
BD_PATH = "Dogs/Bacterial_dermatosis"
FI_PATH = "Dogs/Fungal_infections"
H_PATH = "Dogs/Healthy"
H_A_D_PATH = "Dogs/Hypersensitivity_allergic_dermatosis"
EPOCHS = 40
IMG_SIZE = (128, 128)  
BATCH_SIZE = 32
SEED = 12

# Random seeds for reproducibility
np.random.seed(SEED)
tf.random.set_seed(SEED)

def get_image_paths(folder):
    # Return list of paths to images found in specified folder.
    if not os.path.exists(folder):
        raise FileNotFoundError(f"The directory {folder} does not exist.")
    return sorted([
        Path(folder) / p for p in os.listdir(folder)
        if p.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))
        and not p.startswith('.')
    ])

def load_and_resize_image(path, size=IMG_SIZE):
    # Load image from path and resize it to required size.
    try:
        img = Image.open(path).convert("RGB")
        img = img.resize(size)
        return np.array(img)
    except Exception as e:
        print(f"Error loading image {path}: {e}")
        return None

def load_images_and_labels(paths, label):
    # Load images and labels from specified paths.
    images = [load_and_resize_image(p) for p in paths]
    images = [img for img in images if img is not None]  # Remove failed loads
    labels = [label] * len(images)
    return images, labels

def load_data():
    # Load and prepare the dataset.
    bds_paths = get_image_paths(BD_PATH)
    fi_paths = get_image_paths(FI_PATH)
    h_paths = get_image_paths(H_PATH)
    h_a_d_paths = get_image_paths(H_A_D_PATH)

    bds_images, bds_labels = load_images_and_labels(bds_paths, 0)
    fi_images, fi_labels = load_images_and_labels(fi_paths, 1)
    h_images, h_labels = load_images_and_labels(h_paths, 2)
    h_a_d_images, h_a_d_labels = load_images_and_labels(h_a_d_paths, 3)

    images = np.array(bds_images + fi_images + h_images + h_a_d_images)
    labels = np.array(bds_labels + fi_labels + h_labels + h_a_d_labels)

    return train_test_split(images, labels, test_size=0.2, random_state=SEED, shuffle=True)

def create_model():
    # CNN Network Model.
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        MaxPool2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # 4 classes; change accordingly if we have more classifications
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, # Normalisation
    rotation_range=180, # To capture more different angles
    
    # 15% Tolerance Range
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    
    # Account for Image Flipping
    horizontal_flip=True,
    vertical_flip=True,
    
    fill_mode='nearest'
)

# Load and prepare the dataset
images_train, images_test, labels_train, labels_test = load_data()
model = create_model()

# Train the model
train_generator = train_datagen.flow(
    images_train, labels_train,
    batch_size=BATCH_SIZE  # Adjust batch size
)

# Train the model
model.fit(
    train_generator,
    
    # Matching the steps_per_epoch to Batch size to ensure that the 
    # model is trained on the entire dataset.
    steps_per_epoch= np.ceil(len(images_train) / BATCH_SIZE),  
    
    epochs=EPOCHS,  # Adjust number of epochs
    validation_data=(images_test, labels_test)
)


Epoch 1/40
11/11 [==============================] - 12s 802ms/step - loss: 2.5054 - accuracy: 0.2963 - val_loss: 98.1223 - val_accuracy: 0.2386
Epoch 2/40
11/11 [==============================] - 3s 290ms/step - loss: 1.3564 - accuracy: 0.3305 - val_loss: 70.9601 - val_accuracy: 0.2386
Epoch 3/40
11/11 [==============================] - 3s 242ms/step - loss: 1.3356 - accuracy: 0.3333 - val_loss: 85.0807 - val_accuracy: 0.2841
Epoch 4/40
11/11 [==============================] - 4s 339ms/step - loss: 1.3394 - accuracy: 0.3390 - val_loss: 116.0930 - val_accuracy: 0.2727
Epoch 5/40
11/11 [==============================] - 3s 245ms/step - loss: 1.3066 - accuracy: 0.3590 - val_loss: 114.0874 - val_accuracy: 0.2841
Epoch 6/40
11/11 [==============================] - 3s 286ms/step - loss: 1.2934 - accuracy: 0.3704 - val_loss: 114.9670 - val_accuracy: 0.2727
Epoch 7/40
11/11 [==============================] - 3s 265ms/step - loss: 1.2859 - accuracy: 0.3618 - val_loss: 117.0703 - val_accuracy: 0